<a href="https://colab.research.google.com/github/tanyajais04/copy_moveFrogery.Train/blob/main/code1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras
import math
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
class DataGen(keras.utils.Sequence):
    def __init__(self, ids, path, batch_size=10, image_h=256, image_w = 256):
        self.ids = ids
        self.path = path
        self.batch_size = batch_size
        self.image_h = image_h
        self.image_w = image_w
        self.on_epoch_end()
        
    def __load__(self, id_name):
        ## Path
        image_path = os.path.join(self.path,'train_img', id_name)
        if '.png' in id_name:
            stringforid = id_name.strip('.png')
        if '.jpg' in id_name:
            stringforid = id_name.strip('.jpg')
        stringId = stringforid+'_mask.png'
        
        mask_path = os.path.join(self.path,'train_mask', stringId)      

        ## Reading Imag
        image = cv2.imread(image_path, 1)
        image = cv2.resize(image, (self.image_h, self.image_w))
        mask = np.zeros((self.image_h, self.image_w, 1))
           
        mask_img = cv2.imread(mask_path, 0)
        mask_img = cv2.resize(mask_img, (self.image_h, self.image_w))
        
        mask_img = np.expand_dims(mask_img, axis=-1)
        mask = np.maximum(mask, mask_img)
                
        ## Normalizaing 
        image = image/255.0
        mask = mask/255.0
        
        return image, mask
    
    def __getitem__(self, index):
        if(index+1)*self.batch_size > len(self.ids):
            self.batch_size = len(self.ids) - index*self.batch_size
        
        files_batch = self.ids[index*self.batch_size : (index+1)*self.batch_size]
        
        image = []
        mask  = []
        
        for id_name in files_batch:
            _img, _mask = self.__load__(id_name)
            image.append(_img)
            mask.append(_mask)
            
        image = np.array(image)
        mask  = np.array(mask)
        
        return image, mask
    
    def on_epoch_end(self):
        pass
    
    def __len__(self):
        return int(np.ceil(len(self.ids)/float(self.batch_size)))

In [ ]:
image_h = 256
image_w = 256

img_path = '/content/drive/MyDrive/CoMoFoD/Train'

epochs = 200

batch_size = 10

##Ids
ids = next(os.walk(os.path.join(img_path,'train_img')))[2]
data = DataGen(ids, img_path, image_h=image_h, image_w = image_w, batch_size=batch_size)
train_ids= ids[:3600]
valid_ids = ids[3600:]


In [ ]:
def down_block(x, filters, kernel_size=(3, 3), padding="same", strides=1):  
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides)(x)
    c = keras.layers.Activation('relu')(c)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides)(c)
    c = keras.layers.Activation('relu')(c)
    p = keras.layers.MaxPool2D((2, 2), (2, 2))(c)
    return c, p

def up_block(x, skip, filters, kernel_size=(3, 3), padding="same", strides=1):
    us = keras.layers.UpSampling2D((2, 2))(x)
    concat = keras.layers.Concatenate()([us, skip])
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides)(concat)
    c = keras.layers.Activation('relu')(c)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides)(c)
    c = keras.layers.Activation('relu')(c)
    return c

def bottleneck(x, filters, kernel_size=(5, 5), padding="same", strides=1):
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides)(x)
    c = keras.layers.Activation('relu')(c)
    c = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides)(c)
    c = keras.layers.Activation('relu')(c)

    return c

In [ ]:
def UNet():
    f = [16, 32, 64, 128, 256]
    image_size= 256
    inputs = keras.layers.Input((image_size, image_size, 3))
    
    p0 = inputs
    c1, p1 = down_block(p0, f[0]) #128 -> 64
    c2, p2 = down_block(p1, f[1]) #64 -> 32
    c3, p3 = down_block(p2, f[2]) #32 -> 16
    c4, p4 = down_block(p3, f[3]) #16->8
    
    bn = bottleneck(p4, f[4])
    
    u1 = up_block(bn, c4, f[3]) #8 -> 16
    u2 = up_block(u1, c3, f[2]) #16 -> 32
    u3 = up_block(u2, c2, f[1]) #32 -> 64
    u4 = up_block(u3, c1, f[0]) #64 -> 128
    
    outputs = keras.layers.Conv2D(1, (1, 1), padding="same", activation="sigmoid")(u4)
    model = keras.models.Model(inputs, outputs)
    return model

In [ ]:
model = UNet()
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["acc"])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 16) 448         input_1[0][0]                    
__________________________________________________________________________________________________
activation (Activation)         (None, 256, 256, 16) 0           conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 16) 2320        activation[0][0]                 
______________________________________________________________________________________________

In [ ]:
train_gen = DataGen(train_ids, img_path,image_h=image_h, image_w = image_w , batch_size=batch_size)
valid_gen = DataGen(valid_ids, img_path, image_h=image_h, image_w = image_w, batch_size=batch_size)

train_steps = len(train_ids)//batch_size
valid_steps = len(valid_ids)//batch_size

model.fit_generator(train_gen, validation_data=valid_gen, steps_per_epoch=train_steps, validation_steps=valid_steps, 
                    epochs=epochs)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/200
360/360 [==============================] - 3353s 9s/step - loss: 0.2295 - acc: 0.9431 - val_loss: 0.1710 - val_acc: 0.9503
Epoch 2/200
360/360 [==============================] - 2657s 7s/step - loss: 0.1966 - acc: 0.9442 - val_loss: 0.1727 - val_acc: 0.9512
Epoch 3/200
353/360 [============================>.] - ETA: 48s - loss: 0.1934 - acc: 0.9437

KeyboardInterrupt: ignored